In [ ]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import progressbar
from PIL import Image

In [ ]:
def st_model(content_path='https://github.com/tuttlebr/generative-style-transfer/blob/master/cat_content.jpg',
             style_path='https://github.com/tuttlebr/generative-style-transfer/blob/master/the_fall_of_phaeton.jpg',
             epochs=2,
             steps_per_epoch=200,
             style_weight=0.001,
             content_weight=10000,
             total_variation_weight=100000000):


    def load_img(source, max_dim=512):
        fname = os.path.basename(source)
        destination = os.path.join(os.getcwd(), fname[-40:])
        path_to_img = tf.keras.utils.get_file(destination, source)

        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)

        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img


    def saver(tensorarray):
        save_as = tensorarray.numpy()
        file_name = str(int(tf.timestamp().numpy())) + ".png"
        mpl.image.imsave(file_name, save_as)
        print('\nSaved style transfer image to: {}'.format(file_name))


    def vgg_layers(layer_names):
        """Creates a vgg model that returns a list of intermediate output values."""
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        outputs = [vgg.get_layer(name).output for name in layer_names]

        model = tf.keras.Model([vgg.input], outputs)
        return model


    def gram_matrix(input_tensor):
        """the style of an image can be described by the means and correlations across the different feature maps. 
        Calculate a Gram matrix that includes this information by taking the outer product of the feature vector 
        with itself at each location, and averaging that outer product over all locations."""

        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)


    class StyleContentModel(tf.keras.models.Model):
        def __init__(self, style_layers, content_layers):
            super(StyleContentModel, self).__init__()
            self.vgg =  vgg_layers(style_layers + content_layers)
            self.style_layers = style_layers
            self.content_layers = content_layers
            self.num_style_layers = len(style_layers)
            self.vgg.trainable = False

        def call(self, inputs):
            "Expects float input in [0,1]"
            inputs = inputs*255.0
            preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
            outputs = self.vgg(preprocessed_input)
            style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                              outputs[self.num_style_layers:])

            style_outputs = [gram_matrix(style_output)
                             for style_output in style_outputs]

            content_dict = {content_name:value 
                            for content_name, value 
                            in zip(self.content_layers, content_outputs)}

            style_dict = {style_name:value
                          for style_name, value
                          in zip(self.style_layers, style_outputs)}

            return {'content':content_dict, 'style':style_dict}


    def clip_0_1(image):
        return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)


    def style_content_loss(outputs):
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                               for name in style_outputs.keys()])
        style_loss *= style_weight / num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                                 for name in content_outputs.keys()])
        content_loss *= content_weight / num_content_layers
        loss = style_loss + content_loss
        return loss


    def high_pass_x_y(image):
        """Artifact reduction."""
        x_var = image[:,:,1:,:] - image[:,:,:-1,:]
        y_var = image[:,1:,:,:] - image[:,:-1,:,:]

        return x_var, y_var


    def total_variation_loss(image):
        x_deltas, y_deltas = high_pass_x_y(image)
        return tf.reduce_mean(x_deltas**2) + tf.reduce_mean(y_deltas**2)


    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            loss = style_content_loss(outputs)
            loss += total_variation_weight*total_variation_loss(image)

        grad = tape.gradient(loss, image)
        opt.apply_gradients([(grad, image)])
        image.assign(clip_0_1(image))
    
    def timer(secs):
        mins,secs = divmod(secs, 60)
        hours,mins = divmod(mins, 60)
        return '%02d:%02d:%02d' % (hours, mins, secs)


    content_image = load_img(content_path)
    style_image = load_img(style_path)

    # Content layer with decent results.
    content_layers = ['block5_conv2'] 

    # Style layers with decent results.
    style_layers = ['block1_conv1',
                    'block2_conv1',
                    'block3_conv1', 
                    'block4_conv1', 
                    'block5_conv1']

    num_content_layers = len(content_layers)
    num_style_layers = len(style_layers)
    style_extractor = vgg_layers(style_layers)
    style_outputs = style_extractor(style_image*255)
    extractor = StyleContentModel(style_layers, content_layers)
    results = extractor(tf.constant(content_image))
    style_results = results['style']
    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    image = tf.Variable(content_image)
    opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    image = tf.Variable(content_image)

    step = 0
    bar = progressbar.ProgressBar(maxval=(epochs*steps_per_epoch), \
    widgets=["Style Transfer Progress:", progressbar.Bar('=', '[', ']'), ' ', 
             progressbar.Percentage(), ' ', 
             progressbar.AdaptiveETA()])
    bar.start()
    
    for n in range(epochs):
        for m in range(steps_per_epoch):
            step += 1
            train_step(image)
            bar.update((step-1)+1)
    bar.finish()
    #saver(image[0])
    
    return image[0]

In [ ]:
style_and_content = st_model(epochs=20)

In [ ]:
plt.rcParams["figure.figsize"] = [8,8]
plt.axis('off')
plt.imshow(style_and_content.numpy());